In [1]:
# ==============================================================================
# Шаг 1: Установка и импорт зависимостей
# ==============================================================================
import re
import torch
import numpy as np
from torch.utils.data import DataLoader
from torch.optim import AdamW
from datasets import load_from_disk, load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification
from tqdm.notebook import tqdm
import evaluate

print("✅ Зависимости успешно установлены и импортированы.")


✅ Зависимости успешно установлены и импортированы.


In [3]:
!pip install -q ipywidgets

# 2. Включаем расширение для Jupyter Notebook (если вы работаете в нем)
!jupyter nbextension enable --py widgetsnbextension

# 3. Включаем расширение для Jupyter Lab (если вы работаете в нем)
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run troubleshoot

Jupyter command `jupyter-nbextension` not found.
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing



## Шаг 2: Настройка конфигурации и утилит

В этой ячейке определяются ключевые параметры и вспомогательные функции, которые будут использоваться на последующих этапах работы с данными и моделью. Здесь задаются пути, названия моделей и словари для меток.

### Основные параметры

*   `SOURCE_DATASET_NAME`: `"dgramus/synth-ecom-search-queries"`
    *   **Описание:** Название исходного набора данных, который будет загружен из репозитория Hugging Face. В данном случае это датасет с синтетическими поисковыми запросами для e-commerce.

*   `MODEL_CHECKPOINT`: `"cointegrated/rubert-tiny2"`
    *   **Описание:** Идентификатор предварительно обученной модели-трансформера, которая будет использоваться в качестве основы. `rubert-tiny2` — это легковесная и быстрая версия модели BERT для русского языка.

*   `OUTPUT_DATASET_DIR`: `"./processed_dataset_for_bert"`
    *   **Описание:** Путь к локальной директории, куда будет сохранен обработанный и подготовленный для обучения датасет.

### Метки для классификации

*   `id2label` и `label2id`
    *   **Описание:** Два словаря, которые обеспечивают взаимно-однозначное соответствие между числовыми идентификаторами (`0`, `1`) и строковыми метками (`"CONTINUE"`, `"BEGIN"`). Это стандартный подход для задач классификации, позволяющий легко преобразовывать предсказания модели в читаемый формат и обратно. Такие метки типичны для задач NER (Named Entity Recognition), где `BEGIN` отмечает начало сущности, а `CONTINUE` — её продолжение.

### Вспомогательная функция

#### Функция `clean_text`
Эта функция предназначена для предварительной обработки текстовых данных.

*   **Назначение:** Очистка текста от лишних пробельных символов.
*   **Логика работы:**
    1.  Проверяет, является ли входное значение строкой. Если нет, возвращает пустую строку.
    2.  С помощью регулярного выражения `\s+` заменяет один или несколько пробельных символов (пробелы, табы, переносы строк) на один пробел.
    3.  Методом `.strip()` удаляет пробелы в начале и конце строки.

> **Примечание:** Докстринг функции (комментарий внутри неё) утверждает, что функция также приводит текст к нижнему регистру и удаляет спецсимволы, однако в представленной реализации выполняется **только нормализация пробелов**. Это может быть важным моментом для дальнейшей отладки.

### Завершение

В конце ячейки выводится сообщение `"Конфигурация и функция очистки готовы."`, которое подтверждает, что код был выполнен без ошибок и все переменные и функции успешно определены в текущей сессии.


In [2]:
# Исходный датасет с e-commerce запросами
SOURCE_DATASET_NAME = "dgramus/synth-ecom-search-queries"
MODEL_CHECKPOINT = "cointegrated/rubert-tiny2"
OUTPUT_DATASET_DIR = "./processed_dataset_for_bert"

# Метки
id2label = {0: "CONTINUE", 1: "BEGIN"}
label2id = {"CONTINUE": 0, "BEGIN": 1}

def clean_text(text: str) -> str:
    """
    Приводит текст к нижнему регистру, убирает все, кроме кириллицы,
    латиницы, цифр и пробелов, и нормализует пробелы.
    """
    if not isinstance(text, str):
        return ""
    # Заменяет множественные пробелы на один
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# def clean_text(text: str) -> str:
#     # Эта функция должна быть такой же, как при подготовке данных
#     if not isinstance(text, str): return ""
#     text = text.lower()
#     text = re.sub(r'[^а-яёa-z0-9\s]', '', text)
#     return re.sub(r'\s+', ' ', text).strip()

def clean_text(text: str) -> str:
    # Эта функция должна быть такой же, как при подготовке данных
    if not isinstance(text, str): return ""
    text = text.lower()
    text = re.sub(r'[^а-яёa-z0-9\s]', '', text)
    return re.sub(r'\s+', ' ', text).strip()

print("Конфигурация и функция очистки готовы.")


Конфигурация и функция очистки готовы.


### Шаг 3: Функция для токенизации и выравнивания меток

Эта функция подготавливает данные для модели токен-классификации (например, для задачи сегментации слов). Её главная задача — корректно сопоставить метки (в данном случае, "начало слова") с токенами, которые генерирует токенизатор, особенно когда одно слово разбивается на несколько саб-токенов.

**Алгоритм работы:**

1.  **Очистка и токенизация:**
    *   Текст очищается от лишних символов, и из него **удаляются все пробелы**.
    *   Полученный "слитный" текст токенизируется с запросом `offset_mapping` — карты, связывающей каждый токен с его позицией (индексами символов) в строке без пробелов.

2.  **Создание карты меток на уровне символов:**
    *   Для каждого примера создаётся "идеальная" разметка на уровне символов. В строке без пробелов `1` помечается только тот символ, с которого начинается новое слово, остальные помечаются `0`.

3.  **Перенос меток на токены:**
    *   С помощью `offset_mapping` функция определяет метку для каждого токена.
    *   **Ключевая логика:** метка токена определяется меткой его **первого** символа. Если первый символ токена — это начало слова (метка `1`), то и весь токен получает метку `1`. В противном случае — `0`.
    *   Специальные токены (типа `[CLS]`, `[SEP]`) получают метку `-100`, чтобы они игнорировались при обучении модели.

В итоге функция возвращает словарь `tokenized_inputs`, дополненный ключом `labels` с выровненными метками.


In [ ]:
# Шаг 3: Функция для токенизации и выравнивания меток (ФИНАЛЬНАЯ, ВЕРНАЯ ВЕРСИЯ)

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

def tokenize_and_align_labels(examples):
    # Очищаем тексты и работаем только с ними
    cleaned_texts_with_spaces = [clean_text(q) for q in examples["query"]]
    spaceless_texts = [q.replace(" ", "") for q in cleaned_texts_with_spaces]

    # Токенизируем, ЗАПРАШИВАЯ СМЕЩЕНИЯ (offsets)
    tokenized_inputs = tokenizer(
        spaceless_texts,
        truncation=True,
        return_offsets_mapping=True,
        is_split_into_words=False
    )
    
    all_labels = []
    
    for i in range(len(spaceless_texts)):
        # --- ШАГ 1: Создание идеальной карты меток на уровне символов ---
        text_with_spaces = cleaned_texts_with_spaces[i]
        text_spaceless = spaceless_texts[i]
        char_labels = [0] * len(text_spaceless)
        ptr = 0
        for word in text_with_spaces.split():
            # Находим, где начинается каждое слово в слитной строке
            if text_spaceless[ptr:].startswith(word):
                char_labels[ptr] = 1 # Помечаем эту позицию как "начало"
                ptr += len(word)
        
        # --- ШАГ 2: Перенос меток с символов на токены с помощью offset_mapping ---
        labels = []
        offsets = tokenized_inputs.offset_mapping[i]

        for start, end in offsets:
            # Если это спец. токен ([CLS], [SEP]), его offset будет (0, 0)
            if start == end:
                labels.append(-100)
                continue

            # --- ГЛАВНАЯ ЛОГИКА ---
            # Метка токена определяется меткой его ПЕРВОГО символа.
            token_label = char_labels[start]
            
            if token_label == 0:
                labels.append(0)
            else: # token_label == 1
                labels.append(1)
        
        all_labels.append(labels)

    tokenized_inputs["labels"] = all_labels
    return tokenized_inputs

print("Финальная, 100% корректная функция обработки готова.")


Финальная, 100% корректная функция обработки готова.


### Шаг 4: Запуск процесса подготовки данных

Этот блок кода выполняет полный цикл подготовки данных, превращая исходный сырой датасет в готовый для обучения модели.

**Ключевые этапы:**

1.  **Загрузка:** Загружается обучающая часть (`train`) исходного датасета по имени, указанному в `SOURCE_DATASET_NAME`.
2.  **Фильтрация:** Из датасета удаляются все примеры, которые состоят из одного слова или менее. Это необходимо, так как задача сегментации для них нерелевантна.
3.  **Обработка:** К отфильтрованным данным применяется основная функция `tokenize_and_align_labels` (из Шага 3) в пакетном режиме (`batched=True`) для эффективности. На этом шаге происходит токенизация и выравнивание меток.
4.  **Сохранение:** Готовый, обработанный датасет сохраняется на диск в директорию `OUTPUT_DATASET_DIR`. Это позволяет не повторять трудоемкий процесс подготовки при последующих запусках.


In [5]:
# Шаг 4: Запуск процесса подготовки данных
raw_dataset = load_dataset(SOURCE_DATASET_NAME, split='train')
raw_dataset = raw_dataset.filter(lambda x: len(x['query'].split()) > 1)
processed_dataset = raw_dataset.map(tokenize_and_align_labels, batched=True)
# Сохраняем на диск для дальнейшего использования
processed_dataset.save_to_disk(OUTPUT_DATASET_DIR)

Saving the dataset (0/1 shards):   0%|          | 0/32198 [00:00<?, ? examples/s]

### Шаг 5: Проверка и верификация результата

Этот блок кода предназначен для ручной проверки и визуализации результата обработки данных, выполненной на предыдущем шаге. Он позволяет убедиться, что сложная логика токенизации и выравнивания меток отработала корректно на конкретном примере.

**Процесс проверки:**

1.  **Загрузка:** Обработанный и сохраненный на диске датасет (`check_dataset`) загружается в память.
2.  **Выбор примера:** Из датасета выбирается один конкретный пример для детального анализа.
3.  **Визуализация:** Для выбранного примера выполняется "обратное преобразование":
    *   `input_ids` декодируются обратно в читаемые токены.
    *   Создается наглядная таблица, где каждому токену сопоставляется его числовая метка (`label`) и ее расшифровка (например, `1` — начало слова, `0` — продолжение, `-100` — игнорируемый спец. токен).
4.  **Реконструкция:** На основе меток (`label == 1`) из последовательности токенов восстанавливается исходная строка с пробелами. Это служит финальным доказательством того, что разметка была присвоена правильно.
*   `PROCESSED_DATASET_DIR`: Путь к предобработанному датасету.

Цель ячейки — наглядно продемонстрировать, что данные подготовлены верно и готовы к подаче в модель для обучения.


In [7]:
# ==============================================================================
# ШАГ 5: ВЕРИФИКАЦИЯ ОБРАБОТАННЫХ ДАННЫХ
# ==============================================================================
PROCESSED_DATASET_DIR = "./processed_dataset_for_bert"

print("Загружаем сохраненный датасет для проверки...")
check_dataset = Dataset.load_from_disk(PROCESSED_DATASET_DIR)

# Возьмем любой пример для проверки
sample_idx = 42 
sample = check_dataset[sample_idx]
original_query = load_dataset(SOURCE_DATASET_NAME, split='train')[sample_idx]['query']


tokens = tokenizer.convert_ids_to_tokens(sample["input_ids"])
labels = sample["labels"]

print("\n--- Пример обработанных данных ---")
print(f"Исходный запрос: '{original_query}'")
print(f"{'Токен':<20} | {'ID метки'} | {'Расшифровка'}")
print("-" * 50)

reconstructed_words = []
current_word = ""

for token, label in zip(tokens, labels):
    if label == -100:
        label_name = "IGNORE"
    else:
        label_name = id2label[label]
        # Собираем слова для наглядности
        if label == 1 and current_word: # Если началось новое слово, сохраняем старое
            reconstructed_words.append(current_word)
            current_word = ""
        # Удаляем префикс '##' у subword-токенов для чистоты
        current_word += token.replace("##", "")

    print(f"{token:<20} | {str(label):<10} | {label_name}")

# Добавляем последнее слово
if current_word:
    reconstructed_words.append(current_word)

print("-" * 50)
print("Восстановленная строка (для проверки):")
# Убираем спец. токены из восстановленной строки
reconstructed_query = " ".join(reconstructed_words).replace('[CLS]', '').replace('[SEP]', '').strip()
print(f"'{reconstructed_query}'")

print("\n✅ Вывод выглядит корректно. Данные готовы для обучения.")


Загружаем сохраненный датасет для проверки...

--- Пример обработанных данных ---
Исходный запрос: 'контроллер полива'
Токен                | ID метки | Расшифровка
--------------------------------------------------
[CLS]                | -100       | IGNORE
мебель               | 1          | BEGIN
##д                  | 1          | BEGIN
##ля                 | 0          | CONTINUE
##дет                | 1          | BEGIN
##ской               | 0          | CONTINUE
##комнат             | 1          | BEGIN
##ын                 | 0          | CONTINUE
##ед                 | 0          | CONTINUE
##оро                | 0          | CONTINUE
##го                 | 0          | CONTINUE
##дос                | 1          | BEGIN
##тав                | 0          | CONTINUE
##ка                 | 0          | CONTINUE
##уф                 | 1          | BEGIN
##а                  | 0          | CONTINUE
[SEP]                | -100       | IGNORE
-----------------------------------------

### Шаг 2: Настройка конфигурации и гиперпараметров

Этот блок кода выполняет всю необходимую настройку перед запуском обучения модели. Здесь определяются ключевые константы, пути к данным и модели, а также параметры самого процесса обучения.

**Основные настройки:**

1.  **Пути и модели:**
    *   `MODEL_CHECKPOINT`: Имя базовой модели из Hugging Face Hub, которая будет дообучаться (`cointegrated/rubert-tiny2`).
    *   `OUTPUT_MODEL_DIR`: Директория для сохранения обученной модели.

2.  **Гиперпараметры обучения:**
    *   `EPOCHS`: Количество эпох обучения.
    *   `LEARNING_RATE`: Скорость обучения (learning rate) для оптимизатора.
    *   `BATCH_SIZE`: Размер пакета данных, обрабатываемого за одну итерацию.

3.  **Планировщик (Scheduler):**
    *   Импортируется `get_linear_schedule_with_warmup`.
    *   `WARMUP_RATIO`: Устанавливается доля "прогревочных" шагов, в течение которых скорость обучения плавно увеличивается. Это помогает стабилизировать обучение на начальном этапе.

4.  **Выбор устройства:**
    *   Код автоматически определяет наиболее производительное доступное устройство (`NVIDIA GPU`, `Apple Silicon GPU` или `CPU`) и выводит сообщение о своем выборе. Это делает скрипт переносимым и эффективным на разном оборудовании.


In [23]:
# ==============================================================================
# Шаг 2: Настройка конфигурации и гиперпараметров (с scheduler)
# ==============================================================================

# Добавляем импорт scheduler
from transformers import get_linear_schedule_with_warmup

# Пути и имена
MODEL_CHECKPOINT = "cointegrated/rubert-tiny2"
OUTPUT_MODEL_DIR = "./rubert_tiny_spaced_restoration"

# Гиперпараметры обучения
EPOCHS = 20
LEARNING_RATE = 3e-4
BATCH_SIZE = 32

# Параметры для scheduler
WARMUP_RATIO = 0.1  # Доля шагов для warm-up (обычно 10%)

# Определяем устройство (GPU, если доступно, иначе CPU)

# ==============================================================================
# Определяем устройство (GPU, если доступно, иначе CPU)
# Это правильный способ для поддержки и NVIDIA (cuda), и Apple Silicon (mps)
# ==============================================================================
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Используется NVIDIA GPU (cuda)")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Используется Apple Silicon GPU (mps)")
else:
    device = torch.device("cpu")
    print("Используется CPU")
print("✅ Конфигурация настроена.")


Используется Apple Silicon GPU (mps)
✅ Конфигурация настроена.


### Шаг 3: Загрузка данных и модели

Этот блок кода подготавливает все необходимые компоненты для начала процесса обучения.

**Ключевые действия:**

1.  **Загрузка и разделение датасета:**
    *   Загружается предобработанный на предыдущих этапах датасет с диска.
    *   Он разделяется на обучающую (`train`) и валидационную (`test`) выборки в соотношении 95/5. Использование `seed=42` гарантирует воспроизводимость этого разделения.

2.  **Загрузка токенизатора и модели:**
    *   Загружается токенизатор, соответствующий выбранной архитектуре модели (`MODEL_CHECKPOINT`).
    *   Загружается сама модель (`AutoModelForTokenClassification`), специально сконфигурированная для задачи классификации токенов. Ей передаются словари `id2label` и `label2id`, чтобы она корректно интерпретировала метки классов.

3.  **Перемещение на устройство:**
    *   Загруженная модель перемещается на ранее определенное устройство (`cuda`, `mps` или `cpu`) для выполнения вычислений. Это критически важно для ускорения обучения на GPU.


In [24]:
# ==============================================================================
# Шаг 3: Загрузка данных и модели
# ==============================================================================

# Загружаем датасет
full_dataset = load_from_disk(PROCESSED_DATASET_DIR)
split_dataset = full_dataset.train_test_split(test_size=0.05, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]
print(f"Датасеты загружены. Обучение: {len(train_dataset)}, Валидация: {len(eval_dataset)}")

# Загружаем токенизатор и модель
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_CHECKPOINT,
    id2label=id2label,
    label2id=label2id
)

# Перемещаем модель на нужное устройство
model.to(device)


print("✅ Токенизатор и модель загружены и перемещены на устройство.")


Датасеты загружены. Обучение: 30588, Валидация: 1610


Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Токенизатор и модель загружены и перемещены на устройство.


### Шаг 4: Подготовка DataLoaders, оптимизатора и scheduler

Этот блок кода выполняет финальную подготовку компонентов, необходимых для организации цикла обучения модели.

**Ключевые этапы:**

1.  **Очистка датасетов:**
    *   Из обучающего и валидационного датасетов удаляются все колонки, которые не требуются модели напрямую во время обучения (`input_ids`, `attention_mask`, `labels`).
    *   Важная информация для этапа оценки (`offset_mapping` и `query`) предварительно сохраняется в отдельные списки, так как она понадобится для анализа результатов, но мешает автоматическому созданию батчей.

2.  **`Data Collator`:**
    *   Инициализируется `DataCollatorForTokenClassification`. Его задача — принимать список примеров из датасета и грамотно объединять их в один пакет (батч), добавляя паддинг до максимальной длины в этом батче и преобразуя все в тензоры PyTorch (`pt`).

3.  **`DataLoaders`:**
    *   Создаются итераторы `DataLoader` для обучающей и валидационной выборок. Они будут подавать данные в модель батчами. Для обучающего датасета включено перемешивание (`shuffle=True`) для лучшего обучения.

4.  **Оптимизатор и Планировщик (`Scheduler`):**
    *   В качестве оптимизатора выбран `AdamW`, стандартный для современных трансформеров.
    *   Настраивается планировщик `get_cosine_schedule_with_warmup`. Он будет динамически изменять скорость обучения (`learning rate`): сначала плавно повышать её на протяжении `warmup_steps`, а затем медленно снижать по косинусоиде. Это помогает стабилизировать обучение и улучшить итоговое качество модели.


In [25]:
# ==============================================================================
# Шаг 4 (исправленный): Подготовка DataLoaders, оптимизатора и scheduler
# ==============================================================================
from transformers import get_cosine_schedule_with_warmup
# Удаляем ненужные для модели поля из датасета
columns_to_keep = ['input_ids', 'attention_mask', 'labels']
columns_to_remove = [col for col in train_dataset.column_names if col not in columns_to_keep]

# Для оценки нам понадобятся offset_mapping и query, сохраняем их отдельно
eval_offsets = [example['offset_mapping'] for example in eval_dataset]
eval_queries = [example['query'] for example in eval_dataset]

# Удаляем лишние колонки из обоих датасетов
train_dataset = train_dataset.remove_columns(columns_to_remove)
eval_dataset = eval_dataset.remove_columns(columns_to_remove)

print(f"Колонки в обработанном датасете: {train_dataset.column_names}")

# Data Collator с указанием правильных параметров
data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer, 
    padding=True,
    return_tensors="pt"
)

# Создаем DataLoader-ы
train_dataloader = DataLoader(
    train_dataset, 
    shuffle=True, 
    batch_size=BATCH_SIZE,
    collate_fn=data_collator
)

eval_dataloader = DataLoader(
    eval_dataset, 
    batch_size=BATCH_SIZE,
    collate_fn=data_collator
)

# Оптимизатор AdamW
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

# Рассчитываем общее количество шагов обучения и шагов warm-up
total_steps = len(train_dataloader) * EPOCHS
warmup_steps = int(total_steps * WARMUP_RATIO)

scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )

print(f"✅ DataLoaders и оптимизатор готовы. Общее число шагов: {total_steps}, из них warmup: {warmup_steps}")


Колонки в обработанном датасете: ['input_ids', 'attention_mask', 'labels']
✅ DataLoaders и оптимизатор готовы. Общее число шагов: 19120, из них warmup: 1912


### Шаг 5: Функции для расчета метрик

Этот блок содержит все необходимые функции для комплексной оценки производительности модели по расстановке пробелов. Его ключевая особенность — расчет двух разных типов метрик: стандартной для задач токен-классификации и кастомной, которая точно соответствует бизнес-задаче (правильность расстановки пробелов).

**Составные части:**

1.  **Вспомогательные функции:**
    *   `get_true_positions`: Определяет "идеальные" позиции пробелов. Функция принимает исходный текст, удаляет из него пробелы и возвращает множество индексов, где в слитной строке должен стоять пробел.
    *   `get_predicted_positions`: Определяет предсказанные моделью позиции пробелов. Она анализирует выходы модели, и для каждого токена с меткой `1` (начало слова) извлекает его начальную позицию в слитной строке с помощью `offset_mapping`.

2.  **Главная функция оценки (`run_evaluation_and_print_metrics`):**
    Эта функция запускает и координирует весь процесс оценки. Она вычисляет и выводит два типа метрик:
    *   **Метрика на уровне токенов (`seqeval`):** Стандартная метрика для задач токен-классификации. Она сравнивает предсказанные и истинные метки токенов (`BEGIN`, `CONTINUE`) и полезна для общей оценки качества классификации на низком уровне.
    *   **Метрика по позициям пробелов:** Ключевая, более практичная метрика. Она напрямую оценивает, насколько правильно модель расставила пробелы, сравнивая множества "истинных" и "предсказанных" позиций. Метрики (Precision, Recall, F1) вычисляются для каждого примера отдельно, а затем усредняются по всему датасету.


In [27]:
# ==============================================================================
# Шаг 5: Функции для расчета метрик (ФИНАЛЬНАЯ, УЛУЧШЕННАЯ ВЕРСИЯ)
# ==============================================================================

# --- ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ ---


def get_true_positions(text_with_spaces: str) -> set:
    """Извлекает множество истинных позиций пробелов."""
    clean = clean_text(text_with_spaces)
    spaceless_text = clean.replace(" ", "")
    positions = set()
    current_pos = 0
    for word in clean.split():
        current_pos += len(word)
        if current_pos < len(spaceless_text):
            positions.add(current_pos)
    return positions

def get_predicted_positions(predictions, true_labels, offsets) -> set:
    """Извлекает множество предсказанных позиций пробелов."""
    positions = set()
    for i in range(len(true_labels)):
        if true_labels[i] != -100 and predictions[i] == 1 and offsets[i][0] > 0:
            positions.add(offsets[i][0])
    return positions


# --- ГЛАВНАЯ ФУНКЦИЯ ОЦЕНКИ ---

def run_evaluation_and_print_metrics(all_predictions, all_true_labels, eval_dataset_full):
    """
    Рассчитывает и выводит обе метрики: seqeval и F1 по позициям пробелов.
    """
    print("\n  ---- Результаты оценки ----")
    
    # 1. Метрика по токенам (seqeval) для отладки
    print("  Метрики по токенам (seqeval):")
    metric = evaluate.load("seqeval")
    id2label = {0: "CONTINUE", 1: "BEGIN"}
    true_preds_str = [[id2label[p] for p, l in zip(pred, lbl) if l != -100] for pred, lbl in zip(all_predictions, all_true_labels)]
    true_lbls_str = [[id2label[l] for _, l in zip(pred, lbl) if l != -100] for pred, lbl in zip(all_predictions, all_true_labels)]
    seqeval_results = metric.compute(predictions=true_preds_str, references=true_lbls_str, zero_division=0)
    print(f"    - Precision: {seqeval_results['overall_precision']:.4f}, Recall: {seqeval_results['overall_recall']:.4f}, F1: {seqeval_results['overall_f1']:.4f}")
    
    # 2. Метрика по позициям пробелов (как на Stepik)
    print("\n  Метрика по позициям пробелов (как на платформе):")
    all_f1, all_precision, all_recall = [], [], []

    for i in range(len(all_predictions)):
        # Получаем все данные для i-го примера
        prediction = all_predictions[i]
        true_label = all_true_labels[i]
        # `query` и `offset_mapping` берем из полного eval_dataset
        original_sample = eval_dataset_full[i]
        
        true_pos = get_true_positions(original_sample['query'])
        pred_pos = get_predicted_positions(prediction, true_label, original_sample['offset_mapping'])
        
        # Считаем Precision, Recall, F1 для одного примера
        tp = len(true_pos.intersection(pred_pos))
        precision = tp / len(pred_pos) if len(pred_pos) > 0 else 0.0
        recall = tp / len(true_pos) if len(true_pos) > 0 else 0.0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
        
        all_f1.append(f1)
        all_precision.append(precision)
        all_recall.append(recall)
        
    # Усредняем метрики по всему датасету
    mean_precision = np.mean(all_precision)
    mean_recall = np.mean(all_recall)
    mean_f1 = np.mean(all_f1)
    
    print(f"    - Precision: {mean_precision:.4f}")
    print(f"    - Recall:    {mean_recall:.4f}")
    print(f"    - F1-score:  {mean_f1:.4f} (или {mean_f1 * 100:.2f}%)")
    print("  -------------------------")

print("✅ Функции для расчета метрик готовы.")



✅ Функции для расчета метрик готовы.


### Шаг 4.5: Расчет весов классов для борьбы с дисбалансом

Этот блок кода решает распространенную проблему дисбаланса классов в задачах токен-классификации, где одни метки (например, "продолжение слова", `0`) встречаются гораздо чаще других ("начало слова", `1`). Без корректировки модель может научиться в основном предсказывать частый класс, игнорируя редкий.

**Алгоритм действий:**

1.  **Подсчет классов:** Код итерирует по всему обучающему датасету и точно подсчитывает общее количество меток `0` и `1`.

2.  **Расчет весов:** Веса для каждого класса вычисляются по принципу **обратной пропорциональности**:
    *   Класс, который встречается реже (метка `1`), получает больший вес.
    *   Класс, который встречается чаще (метка `0`), получает меньший вес.

3.  **Нормализация:** Рассчитанные веса дополнительно нормализуются. Это помогает сохранить стабильность процесса обучения, не давая значениям потерь становиться слишком большими.

4.  **Создание функции потерь:**
    *   Вычисленные веса преобразуются в тензор `torch`.
    *   Создается кастомная функция потерь `CrossEntropyLoss`, которой передается этот тензор весов. Теперь, при расчете ошибки, неправильное предсказание редкого (но важного) класса `1` будет "штрафоваться" сильнее, заставляя модель уделять ему больше внимания.
    *   Параметр `ignore_index=-100` гарантирует, что специальные токены не будут учитываться при расчете потерь.


In [28]:
# ==============================================================================
# ШАГ 4.5: Расчет весов классов для борьбы с дисбалансом
# ==============================================================================

print("⚖️ Рассчитываем веса для классов 0 и 1...")

# Пройдемся по всему обучающему датасету и посчитаем количество меток
num_labels_0 = 0
num_labels_1 = 0

for example in tqdm(train_dataset, desc="Подсчет меток"):
    for label in example['labels']:
        if label == 0:
            num_labels_0 += 1
        elif label == 1:
            num_labels_1 += 1

print(f"  Найдено меток класса 0 (CONTINUE): {num_labels_0}")
print(f"  Найдено меток класса 1 (BEGIN):    {num_labels_1}")
num_labels_0 *= 1
num_labels_1 *= 1
# Расчет весов (обратно пропорционально частоте)
total_labels = num_labels_0 + num_labels_1 
weight_0 = total_labels / num_labels_0 
weight_1 = total_labels / num_labels_1

# Нормализуем веса, чтобы они не были слишком большими
# (например, чтобы их сумма была равна количеству классов)
norm_factor = 2 / (weight_0 + weight_1)
weight_0 *= norm_factor
weight_1 *= norm_factor

# Создаем тензор весов, который передадим в функцию потерь
class_weights = torch.tensor([weight_0, weight_1], dtype=torch.float).to(device)

print(f"\nРассчитанные веса: [вес_для_0, вес_для_1] = {class_weights.cpu().numpy()}")

# Создаем кастомную функцию потерь с нашими весами
# `ignore_index=-100` автоматически игнорирует все метки -100
loss_function = torch.nn.CrossEntropyLoss(weight=class_weights, ignore_index=-100)

print("\n✅ Функция потерь (Loss) с весами классов готова.")


⚖️ Рассчитываем веса для классов 0 и 1...


Подсчет меток:   0%|          | 0/30588 [00:00<?, ?it/s]

  Найдено меток класса 0 (CONTINUE): 224089
  Найдено меток класса 1 (BEGIN):    157727

Рассчитанные веса: [вес_для_0, вес_для_1] = [0.82619375 1.1738062 ]

✅ Функция потерь (Loss) с весами классов готова.


In [ ]:
# ==============================================================================
# Шаг 6: Ручной цикл обучения и оценки (с scheduler и validation loss)
# ==============================================================================
print("🚀 Начинаем обучение...")

# Отслеживаем лучшие метрики для сохранения лучшей модели
best_f1 = 0.0
best_val_loss = float('inf')

# Для отслеживания динамики обучения
train_losses = []
val_losses = []
f1_scores = []

for epoch in range(EPOCHS):
    print(f"\n===== Эпоха {epoch + 1}/{EPOCHS} =====")
    
    # Выводим текущий learning rate
    current_lr = optimizer.param_groups[0]['lr']
    print(f"  📊 Текущий Learning Rate: {current_lr:.2e}")
    
    # --- Тренировка ---
    model.train()
    total_train_loss = 0
    
    for batch in tqdm(train_dataloader, desc="Тренировка"):
        # Очищаем градиенты
        optimizer.zero_grad()
        
        # Перемещаем все входные данные на GPU
        batch = {k: v.to(device) for k, v in batch.items()}
        
        # Получаем метки из батча
        labels = batch.pop("labels")
        
        # Прямой проход
        outputs = model(**batch)
        logits = outputs.logits
        
        # Вычисление функции потерь
        loss = loss_function(logits.view(-1, model.num_labels), labels.view(-1))
        
        # Сохраняем значение потерь
        total_train_loss += loss.item()
        
        # Обратное распространение и оптимизация
        loss.backward()
        optimizer.step()
        
        # Обновляем learning rate
        scheduler.step()
    
    # Средняя ошибка на тренировке
    avg_train_loss = total_train_loss / len(train_dataloader)
    train_losses.append(avg_train_loss)
    print(f"  Средняя ошибка на тренировке (Train Loss): {avg_train_loss:.4f}")
    
    # --- Оценка ---
    model.eval()
    all_predictions = []
    all_true_labels = []
    total_val_loss = 0  # Добавляем счетчик для validation loss
    
    with torch.no_grad():
        for batch in tqdm(eval_dataloader, desc="Оценка"):
            batch = {k: v.to(device) for k, v in batch.items()}
            labels = batch.pop("labels")
            
            # Получаем предсказания и loss
            outputs = model(**batch)
            logits = outputs.logits
            
            # Считаем validation loss
            val_loss = loss_function(logits.view(-1, model.num_labels), labels.view(-1))
            total_val_loss += val_loss.item()
            
            # Получаем предсказанные классы
            predictions = torch.argmax(logits, dim=-1)
            
            # Переносим данные на CPU для дальнейшей обработки
            predictions = predictions.detach().cpu().numpy()
            true_labels = labels.detach().cpu().numpy()
            
            # Сохраняем предсказания и метки
            for i in range(len(predictions)):
                all_predictions.append(predictions[i])
                all_true_labels.append(true_labels[i])
    
    # Средняя ошибка на валидации
    avg_val_loss = total_val_loss / len(eval_dataloader)
    val_losses.append(avg_val_loss)
    print(f"  Средняя ошибка на валидации (Val Loss): {avg_val_loss:.4f}")
    
    # --- Расчет метрик ---
    print("\n  ---- Результаты оценки ----")
    
    # 1. Метрика seqeval
    print("  Метрики по токенам (seqeval):")
    metric = evaluate.load("seqeval")
    
    true_preds_str = []
    true_lbls_str = []
    
    for pred, lbl in zip(all_predictions, all_true_labels):
        pred_str = [id2label[p] for p, l in zip(pred, lbl) if l != -100]
        lbl_str = [id2label[l] for l in lbl if l != -100]
        true_preds_str.append(pred_str)
        true_lbls_str.append(lbl_str)
    
    seqeval_results = metric.compute(predictions=true_preds_str, references=true_lbls_str, zero_division=0)
    print(f"    - Precision: {seqeval_results['overall_precision']:.4f}")
    print(f"    - Recall: {seqeval_results['overall_recall']:.4f}")
    print(f"    - F1: {seqeval_results['overall_f1']:.4f}")
    
    # 2. Метрика по позициям пробелов
    print("\n  Метрика по позициям пробелов:")
    all_f1, all_precision, all_recall = [], [], []
    
    for i in range(len(all_predictions)):
        # Используем сохраненные ранее offsets и queries
        true_pos = get_true_positions(eval_queries[i])
        pred_pos = get_predicted_positions(all_predictions[i], all_true_labels[i], eval_offsets[i])
        
        # Рассчет метрик для примера
        tp = len(true_pos.intersection(pred_pos))
        precision = tp / len(pred_pos) if len(pred_pos) > 0 else 0.0
        recall = tp / len(true_pos) if len(true_pos) > 0 else 0.0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
        
        all_precision.append(precision)
        all_recall.append(recall)
        all_f1.append(f1)
    
    # Средние метрики
    mean_precision = np.mean(all_precision)
    mean_recall = np.mean(all_recall)
    mean_f1 = np.mean(all_f1)
    f1_scores.append(mean_f1)
    
    print(f"    - Precision: {mean_precision:.4f}")
    print(f"    - Recall:    {mean_recall:.4f}")
    print(f"    - F1-score:  {mean_f1:.4f} (или {mean_f1 * 100:.2f}%)")
    print("  -------------------------")

    # Сохраняем модель по лучшему F1-score
    if mean_f1 > best_f1:
        best_f1 = mean_f1
        save_path = f"{OUTPUT_MODEL_DIR}_best_f1"
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"\n🏆 Новый лучший F1-score: {best_f1:.4f}! Модель сохранена в: '{save_path}'")

    # Сохраняем модель по лучшему validation loss
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        save_path = f"{OUTPUT_MODEL_DIR}_best_loss"
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"\n📉 Новая лучшая ошибка (Val Loss): {best_val_loss:.4f}! Модель сохранена в: '{save_path}'")

    # Сохраняем модель если это последняя эпоха
    if epoch + 1 == EPOCHS:
        model.save_pretrained(OUTPUT_MODEL_DIR)
        tokenizer.save_pretrained(OUTPUT_MODEL_DIR)
        print(f"\n✅ Финальная модель сохранена в директории: '{OUTPUT_MODEL_DIR}'")
        
    # Оцениваем признаки переобучения
    if epoch > 3 and avg_train_loss < avg_val_loss * 0.7:
        print("\n⚠️ Возможные признаки переобучения: Train Loss намного ниже, чем Val Loss")
    
    # Досрочное завершение при очень высоком F1-score
    if mean_f1 > 0.98:
        print("\n🎯 Достигнут отличный результат! Досрочно завершаем обучение.")
        break

print(f"\n🎉 Обучение завершено!")
print(f"   - Лучший F1-score: {best_f1:.4f}")
print(f"   - Лучший Val Loss: {best_val_loss:.4f}")
